# Data aggregation

Data aggregation iis the process where raw data is gathered and expressed in a summary from for statistical analysis.

For example. raw data can be aggregated over a given time period to provide statistics such as average, minimum, maximum, sum, and count. After the data is aggregated and written to a view or report you can analyze the aggregated data to gain insights about particular i resources or resource groups.

In [12]:
import pandas as pd
import numpy as np

In [13]:
numbers = pd.Series(np.random.randint(low=1, high=100, size=5))

In [19]:
# Some of Pandas aggregation methods are:
print(f"{numbers.min() = }")
print(f"{numbers.max() = }")
print(f"{numbers.median() = }")
print(f"{numbers.mean() = }")
print(f"{numbers.sum() = }")
print(f"{numbers.sum() / numbers.count() = }")
print(f"{numbers.mode() = }")
print(f"{numbers.count() = }")
print(f"{numbers.size = }")


numbers.min() = 10
numbers.max() = 84
numbers.median() = 65.0
numbers.mean() = 52.2
numbers.sum() = 261
numbers.sum() / numbers.count() = 52.2
numbers.mode() = 0    10
1    28
2    65
3    74
4    84
dtype: int32
numbers.count() = 5
numbers.size = 5


### When run on a dataframe (multiple Series) they return a single value for each Series , forming a new Series

In [21]:
numbers_df = pd.DataFrame(np.random.randint(low=1, high=100, size=[5, 5], ))
numbers_df.loc[[0, 3], [3, 4]] = np.nan
numbers_df

,0,1,2,3,4
0,94,84,92,NaN,NaN
1,57,76,75,47.0,67.0
2,98,89,29,78.0,48.0
3,40,14,6,NaN,NaN
4,21,13,72,93.0,28.0


In [17]:
numbers_df.min(axis="columns")
numbers_df.min(axis="columns").min()
numbers_df.min(axis="index").median()
numbers_df.min(axis="columns").median()

20.0

In [23]:
# Count null values in each column
numbers_df.isna().sum(axis="index")

0    0
1    0
2    0
3    2
4    2
dtype: int64

### Working with real data

In [24]:
autos = pd.read_json("../Data/autos.json")
autos.tail(3)

FileNotFoundError: File ../Data/autos.json does not exist

In [25]:
autos["price"].mean()

NameError: name 'autos' is not defined

In [26]:
autos[["length", "width", "height"]].head(3)

NameError: name 'autos' is not defined

In [27]:
# Running mean() on multiple columns (DataFrame) returns a Series of means.
autos[["length", "width", "height"]].head(3).mean()
autos[["length", "width", "height"]].mean()
autos[autos["make"] == "volvo"][["length", "width", "height"]].mean()
autos.query("make=='volvo'")[["length", "width", "height"]].mean()

NameError: name 'autos' is not defined

### Multiple aggregation

In [28]:
autos[["length", "width", "height"]].agg("max")

NameError: name 'autos' is not defined

In [29]:
autos[["length", "width", "height"]].agg(["min", "mean", "max"])
#autos.info()
#autos.agg(["min", "mean", "max"])
[col for col in autos.columns if autos[col].dtype in["int64", "float64"]]
#{col: autos[col].dtype for col in autos.columns if autos[col].dtype in["int64", "float64"]} #dictionary form

NameError: name 'autos' is not defined

In [30]:
autos.describe()

NameError: name 'autos' is not defined

### Split-Apply-Combine
- Select a feature to use as key
- Split the dataset into group for each unique key value 
- Apply aggregation to each group
- Combine aggregated data into a new dataset
![image](https://nicholasvadivelu.com/assets/images/posts/groupby/split-apply-combine.svg#center)

### Group by
Use Pandas .groupby() method to select a key and split into groups.

This creates a new DataFrameGroupBy object containing the grouped DataFramess

In [31]:
makes = autos.groupby("make")

NameError: name 'autos' is not defined

In [32]:
type(makes)

NameError: name 'makes' is not defined

In [33]:
#autos["make"].unique()
#autos["make"].nunique()
#autos["make"].value_counts()
len(makes)

NameError: name 'makes' is not defined

In [34]:
makes.groups
autos.loc[makes.groups["jaguar"]]
makes.get_group(["jaguar"])


NameError: name 'makes' is not defined

In [36]:
makes.count().head(3)
makes[["length", "width", "height"]].mean().head(3)

NameError: name 'makes' is not defined

### SeriesGroupBy
Indexing a DataFrameGroupBy object with a single column will return a SeriesGroupBy object

In [ ]:
sgb = makes["price"]
type(sgb)
sgb.groups 
sgb.get_group("jaguar")
sgb.mean().head(3)

autos.groupby("make")["price"].mean()

### Multiple aggregations on SeriesGroupBy
Use Pandas .agg() method on SeriesGroupBy to do multiple aggregation on a single feature

In [1]:
sgb.min()
sgb.agg(["min", "mean", "max"])
autos.groupby("make")["price"].agg(["min", "mean", "max"])
autos.groupby("make")["price"].describe().transpose()

NameError: name 'sgb' is not defined

### Multiple aggregation on DataFrameGroupBy

Using pandas .agg() method on a DataFrameGroupBy to do multiple aggregation on multiple features will return a multi-index column dataframe.

In [2]:
makes[["length", "width", "height"]].agg(["min", "mean", "max"])

NameError: name 'makes' is not defined

### Custom columns aggregation

In [4]:
autos.groupby("make").agg({"price": "mean", "horsepower": "max"})

SyntaxError: invalid syntax (982817112.py, line 1)

In [1]:
autos.groupby("make").agg(
    average_price = pd.NamedAgg(column="price", aggfunc="mean")
    min_horsepower = pd.NamedAgg(column="horsepower", aggfunc="min")
    max_horsepower = pd.NamedAgg(column="horsepower", aggfunc="max")
).head(3)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1023709184.py, line 2)

In [ ]:
def list_unique(x):
    return ", ".join(x)

autos.groupby("make").agg(
    average_price = pd.NamedAgg(column="price", aggfunc="mean")
    min_horsepower = pd.NamedAgg(column="horsepower", aggfunc="min")
    max_horsepower = pd.NamedAgg(column="horsepower", aggfunc="max")
    body_styles = pd.NamedAgg(column="body-style", aggfunc=list_unique)
).sort_valuew(by="average_price", ascending=False)

In [4]:
def my_sorting_func(person):
    return person["Age"]

mylist = [
    {"FirstName": "Fredrik", "Lastname": "Johansson", "Age": 42},
    {"FirstName": "Anders", "Lastname": "Svensson", "Age": 22},
    {"FirstName": "Maria", "Lastname": "Karlsson", "Age": 27},
    {"FirstName": "Anna", "Lastname": "Bengtsson", "Age": 39},
    
]
sorted(mylist, key=lambda person: len(person["FirstName"]), reverse=True)

[{'FirstName': 'Fredrik', 'Lastname': 'Johansson', 'Age': 42},
 {'FirstName': 'Anders', 'Lastname': 'Svensson', 'Age': 22},
 {'FirstName': 'Maria', 'Lastname': 'Karlsson', 'Age': 27},
 {'FirstName': 'Anna', 'Lastname': 'Bengtsson', 'Age': 39}]

In [8]:
def myfunc():
    return "Hello"

myfunc = lambda: "Hello"

otherfunc = myfunc

print(otherfunc())

Hello
